In [435]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

In [436]:
df  = pd.read_csv('breast.csv')

In [437]:
# Define X, y:
X = df.drop(columns = 'class')
y = df.iloc[:, -1]

In [438]:
#Separate Data, based on classes
class_labels = np.unique(y)
print(class_labels)

[2 4]


# Using MLE

In [439]:
def separate_classes(class_labels, X, y):
    classes = {}
    for idx, c in enumerate(class_labels):
        X_c = X[y == c]
        if c not in classes:
            classes[c] = []
        classes[c].append(X_c)

    return classes

In [440]:
#Define class probability for each class
def class_prob(X_train, classes):
    N, D = X_train.shape
    class_prob = {}
    for i, instances in classes.items():
        class_prob[i] = len(instances[0]) / N
    return class_prob

In [441]:
#calculate conditional probability
def con_prob(classes):
    conditional_prob = {}
    for c, instances in classes.items():
        features = [list(feature) for feature in zip(*instances)]
        for value in features:
            number_of_xi = instances[0][value].value_counts()[1]
            class_con_prob = number_of_xi / len(instances[0])
            if c not in conditional_prob:
                conditional_prob[c] = []
            conditional_prob[c].append(class_con_prob) 
        
    return conditional_prob

In [442]:
#predict posterior on test data
def naive_bayes_classifier(class_prob, con_prob, test_data):
    best_class = []
    for i, instance in enumerate(test_data.values):
        instance_posterior = {}
        for c, prior in class_prob.items():
            posterior = prior
            for idx, x in enumerate(instance):
                if(x == 1):
                    posterior *= con_prob[c][idx]
            instance_posterior[c] = posterior
        best_class.append(max(instance_posterior))
    return best_class

In [447]:
accuracy = {}
for i in range(20):
    # Split Data to 20% test and 80% training data
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)
    
    #calculate conditional_probability
    classes = separate_classes(class_labels, X_train, y_train)
    class_probability = class_prob(X_train, classes)
    conditional_probability = con_prob(classes)
    best_class = naive_bayes_classifier(class_probability, conditional_probability, X_test)
    #calculate accuracy
    y_true = y_test.values
    y_pred = best_class
    acc = np.sum(y_pred == y_true)/len(y_true)
    accuracy[i] = acc

In [448]:
print("Accurecies: \n", accuracy)

Accurecies: 
 {0: 0.36496350364963503, 1: 0.35036496350364965, 2: 0.291970802919708, 3: 0.36496350364963503, 4: 0.40145985401459855, 5: 0.29927007299270075, 6: 0.36496350364963503, 7: 0.36496350364963503, 8: 0.32116788321167883, 9: 0.34306569343065696, 10: 0.35036496350364965, 11: 0.43795620437956206, 12: 0.36496350364963503, 13: 0.3722627737226277, 14: 0.30656934306569344, 15: 0.3722627737226277, 16: 0.31386861313868614, 17: 0.35766423357664234, 18: 0.3795620437956204, 19: 0.3284671532846715}


In [449]:
#define mean of accuracy
mean_value = sum(accuracy.values()) / len(accuracy)
print("Mean:", mean_value)

Mean: 0.35255474452554747


# Using MAP Estimation

In [450]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Define Beta prior distribution parameters for conditional probabilities
alpha = 1 
beta = 2   

In [451]:
# Separate training data based on class labels
classes = separate_classes(class_labels, X_train, y_train)

In [452]:
#Define class probability for each class
def bera_class_prob(X_train, classes, alpha, beta):
    N, D = X_train.shape
    class_prob = {}
    for i, instances in classes.items():
        class_prob[i] = (len(instances[0]) + alpha) / (N + alpha + beta)
    return class_prob

In [453]:
# Estimate conditional probabilities for each feature given class using MAP with Beta prior
def Beta_dist_con_prob(calsses, alpha, beta):
    conditional_prob = {}
    for c, instances in classes.items():
        features = [list(feature) for feature in zip(*instances)]
        for value in features:
            number_of_xi = instances[0][value].value_counts()[1] + alpha
            class_con_prob = number_of_xi / (len(instances[0]) + alpha + beta)
            if c not in conditional_prob:
                conditional_prob[c] = []
            conditional_prob[c].append(class_con_prob) 
    return conditional_prob

In [454]:
#predict posterior on test data
def beta_naive_bayes_classifier(class_prob, con_prob, test_data):
    best_class = []
    for i, instance in enumerate(test_data.values):
        instance_posterior = {}
        for c, prior in class_prob.items():
            posterior = prior
            for idx, x in enumerate(instance):
                if(x == 1):
                    posterior *= con_prob[c][idx]
            instance_posterior[c] = posterior
        best_class.append(max(instance_posterior))
    return best_class

In [455]:
#calculate conditional_probability on training data
classes = separate_classes(class_labels, X_train, y_train)
class_probability = bera_class_prob(X_train, classes, alpha, beta)
conditional_probability = Beta_dist_con_prob(classes, alpha, beta)

In [456]:
print("class probability: \n", class_probability)

class probability: 
 {2: 0.6666666666666666, 4: 0.331511839708561}


In [457]:
#predict on test data   
map_best_class = beta_naive_bayes_classifier(class_probability, conditional_probability, X_test)

In [458]:
#calculate accuracy
y_true = y_test.values
y_pred = map_best_class
map_acc = np.sum(y_pred == y_true)/len(y_true)

In [460]:
print("Error: \n", 1 - map_acc)

Error: 
 0.5766423357664234


# Report

1- first of all read the dataset and define class labels

2- after that we nead to seperate train data into defferent classes based on class labels, i did this with function separate_classes

3- then for each classes we nead to calculate class probability(θC) which function class_prob do this for us

4- after that the function con_prob calculate conditional probability (θA1∣C) given class y = 2 and y =4 for each feature we have (9 featurs) 

5- then seperate data into training and test data for 20 times and calculate bets class prediction for each instances with function naive_bayes_classifier

6- the function naive_bayes_classifier, calculates θC*θA1∣C for every instance given y=2 and y=4 and choose the highest probability to predict best class for every point in test data

7- then calculate all accuracy for all times we did this process and report the mean of them